In [0]:
foldername="superusersdata1"
filename="animals.csv"
dbutils.fs.mkdirs(foldername)
filepath=foldername+"/"+filename
data="Id,Navn\n1,Hest\n2,Kat\n3,Hund"
dbutils.fs.put(filepath,data,True)

Wrote 27 bytes.
Out[10]: True

In [0]:
print(dbutils.fs.head(filepath))

Id,Navn
1,Hest
2,Kat
3,Hund

In [0]:
%sql
DROP TABLE animalsTable

In [0]:
# File location and type
file_location = "/superusersdata1/animals.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

permanent_table_name = "animalsTable"

#df.write.format("parquet").saveAsTable(permanent_table_name)
#df.write.format("delta").saveAsTable(permanent_table_name)
# default format er delta nu i den spark version vi kører her
df.write.saveAsTable(permanent_table_name)


In [0]:
%sql
SELECT * FROM animalsTable

Id Navn 1 Hest 2 Kat 3 Hund

In [0]:
%sql
INSERT INTO animalsTable
VALUES(4,'Giraf')

num_affected_rows num_inserted_rows 1 1

In [0]:
%sql
DELETE FROM animalsTable
WHERE Id=4

num_affected_rows 1

In [0]:
%sql
UPDATE animalsTable
SET Navn='Flodhest'
WHERE Id=1

num_affected_rows 1

In [0]:
%sql
DESCRIBE TABLE EXTENDED animalsTable

col_name data_type comment Id int Navn string # Partitioning Not partitioned # Detailed Table Information Name default.animalstable Location dbfs:/user/hive/warehouse/animalstable Provider delta Owner root Type MANAGED Table Properties [delta.minReaderVersion=1,delta.minWriterVersion=2]

In [0]:
%sql
DROP TABLE animalsTable

In [0]:
%sql
DESCRIBE history animalsTable

version timestamp userId userName operation operationParameters job notebook clusterId readVersion isolationLevel isBlindAppend operationMetrics userMetadata 4 2022-03-03T12:15:10.000+0000 1854991049842989 su20220228@outlook.com RESTORE Map(version -> 1, timestamp -> null) null List(1648024987646919) 0228-114732-tlvbhl6d 3 Serializable false Map(numRestoredFiles -> 2, removedFilesSize -> 742, numRemovedFiles -> 1, restoredFilesSize -> 1470, numOfFilesAfterRestore -> 2, tableSizeAfterRestore -> 1470) null 3 2022-03-03T12:10:08.000+0000 1854991049842989 su20220228@outlook.com UPDATE Map(predicate -> (Id#85409 = 1)) null List(1648024987646919) 0228-114732-tlvbhl6d 2 WriteSerializable false Map(numRemovedFiles -> 1, numCopiedRows -> 2, numAddedChangeFiles -> 0, executionTimeMs -> 1082, scanTimeMs -> 648, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 434) null 2 2022-03-03T12:09:48.000+0000 1854991049842989 su20220228@outlook.com DELETE Map(predicate -> ["(spark_catalog.default.animalsTable.`Id` = 4)"]) null List(1648024987646919) 0228-114732-tlvbhl6d 1 WriteSerializable false Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 638, numDeletedRows -> 1, scanTimeMs -> 422, numAddedFiles -> 0, rewriteTimeMs -> 216) null 1 2022-03-03T12:09:40.000+0000 1854991049842989 su20220228@outlook.com WRITE Map(mode -> Append, partitionBy -> []) null List(1648024987646919) 0228-114732-tlvbhl6d 0 WriteSerializable true Map(numFiles -> 1, numOutputBytes -> 740, numOutputRows -> 1) null 0 2022-03-03T12:09:24.000+0000 1854991049842989 su20220228@outlook.com CREATE TABLE AS SELECT Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {}) null List(1648024987646919) 0228-114732-tlvbhl6d null WriteSerializable true Map(numFiles -> 1, numOutputBytes -> 730, numOutputRows -> 3) null

In [0]:
%sql
SELECT * FROM animalsTable

Id Navn 4 Giraf 1 Hest 2 Kat 3 Hund

In [0]:
%sql
--https://docs.databricks.com/spark/latest/spark-sql/language-manual/delta-restore.html
RESTORE TABLE animalsTable TO VERSION AS OF 1

table_size_after_restore num_of_files_after_restore num_removed_files num_restored_files removed_files_size restored_files_size 1470 2 1 2 742 1470

In [0]:
%fs ls dbfs:/superusersdata1/

path name size dbfs:/superusersdata1/animals.csv animals.csv 27 dbfs:/superusersdata1/animals2.csv animals2.csv 32

In [0]:
%sql
DROP TABLE IF EXISTS animals;

CREATE TABLE animals
USING csv
OPTIONS (path "dbfs:/superusersdata1/*.csv", header "true")

In [0]:
%sql
DESCRIBE TABLE EXTENDED animals

col_name data_type comment Id string null Navn string null # Detailed Table Information Database default Table animals Owner root Created Time Thu Mar 03 12:19:41 UTC 2022 Last Access UNKNOWN Created By Spark 3.1.2 Type EXTERNAL Provider csv Location dbfs:/superusersdata1/*.csv Serde Library org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe InputFormat org.apache.hadoop.mapred.SequenceFileInputFormat OutputFormat org.apache.hadoop.hive.ql.io.HiveSequenceFileOutputFormat Storage Properties [header=true]

In [0]:
%sql
SELECT *
FROM   animals

Id Navn 5 Flodhest 6 Tiger 7 And 1 Hest 2 Kat 3 Hund

In [0]:
foldername="superusersdata1"
filename="animals2.csv"
dbutils.fs.mkdirs(foldername)
filepath=foldername+"/"+filename
data="Id,Navn\n5,Flodhest\n6,Tiger\n7,And"
dbutils.fs.put(filepath,data,True)

Wrote 32 bytes.
Out[19]: True

In [0]:
%sql
REfRESH TABLE animals